In [ ]:
import libceed
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
tab10 = matplotlib.cm.get_cmap('tab10')
BLUE = tab10(0.05)
GREY = tab10(0.75)
YELLOW = tab10(0.85)

In [ ]:
ceed = libceed.Ceed()

max_degree = 8
differences_perturb = np.zeros(max_degree)

for p in range(1, max_degree + 1):
    q = p + 1
    b = ceed.BasisTensorH1Lagrange(1, 1, p + 1, q, libceed.GAUSS)
    interp = b.get_interp_1d().reshape(q, p + 1)
    grad = b.get_grad_1d().reshape(q, p + 1)
    w = np.diag(b.get_q_weight_1d())
    
    # peturb to form pesudoinverse 
    M = interp.T @ w @ interp
    K = grad.T   @ w @ grad
    K = K + np.eye(p + 1)*1e-4
    X, lam = ceed.simultaneous_diagonalization(K, M, p + 1)
    K = grad.T   @ w @ grad
    X = X.reshape(p + 1, p + 1)
    
    lam_pinv = np.diag([1./val for val in lam])
    K_pinv = X @ lam_pinv @ X.T

    A = K
    A_pinv = K_pinv

    left = np.zeros(p + 1)
    left[0] = 1.
    right = np.zeros(p + 1)
    right[p] = 1.
    S = -np.array([np.matmul(A_pinv, left)[0], np.matmul(A_pinv, right)[0],
                   np.matmul(A_pinv, left)[p], np.matmul(A_pinv, right)[p]]).reshape(2, 2)

    S_inv = np.linalg.inv(S)
    
    [nodes, _] = ceed.lobatto_quadrature(p + 1)
    u = np.array([-(node - 1.)*(node + 1.) for node in nodes])
    
    v = np.matmul(A, u)
    
    v[0] = 0
    v[p] = 0
    w = np.matmul(A_pinv, v)
  
    w_pi = [w[0], w[p]]
    v_pi = np.matmul(S_inv, w_pi)
    v[0] = v_pi[0]
    v[p] = v_pi[1]
    
    w = np.matmul(A_pinv, v)
    
    differences_perturb[p - 1] = np.linalg.norm(u - w)

print(differences_perturb)

In [ ]:
ceed = libceed.Ceed()

max_degree = 8
differences_zero = np.zeros(max_degree)

for p in range(1, max_degree + 1):
    q = p + 1
    b = ceed.BasisTensorH1Lagrange(1, 1, p + 1, q, libceed.GAUSS)
    interp = b.get_interp_1d().reshape(q, p + 1)
    grad = b.get_grad_1d().reshape(q, p + 1)
    w = np.diag(b.get_q_weight_1d())
    
    # zero inverse of zero to form pesudoinverse 
    M = interp.T @ w @ interp
    K = grad.T   @ w @ grad
    X, lam = ceed.simultaneous_diagonalization(K, M, p + 1)
    K = grad.T   @ w @ grad
    X = X.reshape(p + 1, p + 1)
    
    lam_pinv = np.diag([1./val if abs(val) > 1e-12 else 0 for val in lam])
    K_pinv = X @ lam_pinv @ X.T

    A = K
    A_pinv = K_pinv

    left = np.zeros(p + 1)
    left[0] = 1.
    right = np.zeros(p + 1)
    right[p] = 1.
    S = -np.array([np.matmul(A_pinv, left)[0], np.matmul(A_pinv, right)[0],
                   np.matmul(A_pinv, left)[p], np.matmul(A_pinv, right)[p]]).reshape(2, 2)

    S_inv = np.linalg.inv(S)
    
    [nodes, _] = ceed.lobatto_quadrature(p+1)
    u = np.array([-(node - 1.)*(node + 1.) for node in nodes])
    
    v = np.matmul(A, u)
    
    v[0] = 0
    v[p] = 0
    w = np.matmul(A_pinv, v)
  
    w_pi = [w[0], w[p]]
    v_pi = np.matmul(S_inv, w_pi)
    v[0] = v_pi[0]
    v[p] = v_pi[1]
    
    w = np.matmul(A_pinv, v)
    
    differences_zero[p - 1] = np.linalg.norm(u - w)

print(differences_zero)

In [ ]:
# Plot 1D pesudoinverse error
fig, ax1 = plt.subplots(figsize=(10, 8))

plt.xlim([1, 8])

plt.plot(range(1, 9), differences_zero, label='Zeroing', marker='s', markersize=10, color=BLUE)
plt.plot(range(1, 9), differences_perturb, label='Perturbation', marker='o', markersize=10, color=YELLOW)

ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.set_yscale('log')
ax1.grid(color='lightgrey')

plt.legend(fontsize=20)
plt.legend(fontsize=20, loc=[0.63,0.45])
plt.tick_params(labelsize=20)
plt.title('1D Subdomain Solver Error', fontsize=26)
plt.ylabel('2-Norm of Error', fontsize=24)
plt.xlabel('Polynomial Order', fontsize=24)
plt.savefig("subdomainError1D", bbox_inches='tight')

In [ ]:
ceed = libceed.Ceed()

max_degree = 8
differences_perturb = np.zeros(max_degree)

for p in range(1, max_degree + 1):
    q = p + 1
    b = ceed.BasisTensorH1Lagrange(1, 1, p + 1, q, libceed.GAUSS)
    interp = b.get_interp_1d().reshape(q, p + 1)
    grad = b.get_grad_1d().reshape(q, p + 1)
    w = np.diag(b.get_q_weight_1d())
    
    # perturb to form pesudoinverse
    M = interp.T @ w @ interp
    K = grad.T   @ w @ grad
    K += np.eye(p + 1)*1e-6
    
    X, lam = ceed.simultaneous_diagonalization(K, M, p + 1)
    K = grad.T   @ w @ grad
    X = X.reshape(p + 1, p + 1)

    M_2 = np.kron(M, M)
    K_2 = np.kron(K, M) + np.kron(M, K)

    M_inv = np.kron(X, X) @ np.kron(X.T, X.T)

    ones = np.ones(p + 1)
    lam_2 = np.kron(ones, lam) + np.kron(lam, ones)
    lam_2_pinv = [1./val for val in lam_2]
    lam_2_pinv = np.diag(lam_2_pinv)
    K_pinv = np.kron(X, X) @ lam_2_pinv @ np.kron(X.T, X.T)

    A = K_2
    A_pinv = K_pinv

    num_vertices = 2**2
    S = np.zeros((num_vertices, num_vertices))
    indx = [0, p, (p + 1)**2 - (p + 1), (p + 1)**2 - 1]
    for j in range(num_vertices):
        u = np.zeros((p + 1)**2)
        u[indx[j]] = 1
        v = np.matmul(A_pinv, u)
        for i in range(num_vertices):
            S[i, j] = -v[indx[i]]

    S_inv = np.linalg.inv(S)
    
    [nodes, _] = ceed.lobatto_quadrature(p + 1)
    u = np.array([-(node_i - 1.)*(node_i + 1.)-(node_j - 1.)*(node_j + 1.) for node_i in nodes for node_j in nodes])

    
    v = np.matmul(A, u)

    for i in range(num_vertices):
        v[indx[i]] = 0
    w = np.matmul(A_pinv, v)

    w_pi = np.zeros(num_vertices)
    for i in range(num_vertices):
        w_pi[i] = w[indx[i]]
    v_pi = np.matmul(S_inv, w_pi)
    for i in range(num_vertices):
        v[indx[i]] = v_pi[i]
    
    w = np.matmul(A_pinv, v)
    
    differences_perturb[p - 1] = np.linalg.norm(u - w)

print(differences_perturb)

In [ ]:
ceed = libceed.Ceed()

max_degree = 8
differences_zero = np.zeros(max_degree)

for p in range(1, max_degree + 1):
    q = p + 1
    b = ceed.BasisTensorH1Lagrange(1, 1, p + 1, q, libceed.GAUSS)
    interp = b.get_interp_1d().reshape(q, p + 1)
    grad = b.get_grad_1d().reshape(q, p + 1)
    w = np.diag(b.get_q_weight_1d())
    
    # zero eigenvalue inverses to form pesudoinverse
    M = interp.T @ w @ interp
    K = grad.T   @ w @ grad
    
    X, lam = ceed.simultaneous_diagonalization(K, M, p + 1)
    K = grad.T   @ w @ grad
    X = X.reshape(p + 1, p + 1)

    M_2 = np.kron(M, M)
    K_2 = np.kron(K, M) + np.kron(M, K)

    M_inv = np.kron(X, X) @ np.kron(X.T, X.T)

    ones = np.ones(p + 1)
    lam_2 = np.kron(ones, lam) + np.kron(lam, ones)
    lam_2_pinv = [1./val if abs(val) > 1e-12 else 0 for val in lam_2]
    lam_2_pinv = np.diag(lam_2_pinv)
    K_pinv = np.kron(X, X) @ lam_2_pinv @ np.kron(X.T, X.T)

    A = K_2
    A_pinv = K_pinv

    num_vertices = 2**2
    S = np.zeros((num_vertices, num_vertices))
    indx = [0, p, (p + 1)**2 - (p + 1), (p + 1)**2 - 1]
    for j in range(num_vertices):
        u = np.zeros((p + 1)**2)
        u[indx[j]] = 1
        v = np.matmul(A_pinv, u)
        for i in range(num_vertices):
            S[i, j] = -v[indx[i]]

    S_inv = np.linalg.inv(S)
    
    [nodes, _] = ceed.lobatto_quadrature(p + 1)
    u = np.array([-(node_i - 1.)*(node_i + 1.)-(node_j - 1.)*(node_j + 1.) for node_i in nodes for node_j in nodes])

    
    v = np.matmul(A, u)

    for i in range(num_vertices):
        v[indx[i]] = 0
    w = np.matmul(A_pinv, v)

    w_pi = np.zeros(num_vertices)
    for i in range(num_vertices):
        w_pi[i] = w[indx[i]]
    v_pi = np.matmul(S_inv, w_pi)
    for i in range(num_vertices):
        v[indx[i]] = v_pi[i]
    
    w = np.matmul(A_pinv, v)
    
    differences_zero[p - 1] = np.linalg.norm(u - w)

print(differences_zero)

In [ ]:
# Plot 2D pesudoinverse error
fig, ax1 = plt.subplots(figsize=(10, 8))

plt.xlim([1, 8])

plt.plot(range(1, 9), differences_zero, label='Zeroing', marker='s', markersize=10, color=BLUE)
plt.plot(range(1, 9), differences_perturb, label='Perturbation', marker='o', markersize=10, color=YELLOW)

ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.set_yscale('log')
ax1.grid(color='lightgrey')

plt.legend(fontsize=20)
plt.legend(fontsize=20, loc=[0.63,0.45])
plt.tick_params(labelsize=20)
plt.title('2D Subdomain Solver Error', fontsize=26)
plt.ylabel('2-Norm of Error', fontsize=24)
plt.xlabel('Polynomial Order', fontsize=24)
plt.savefig("subdomainError2D", bbox_inches='tight')

In [ ]:
ceed = libceed.Ceed()

max_degree = 8
differences_perturb = np.zeros(max_degree)

for p in range(1, max_degree + 1):
    q = p + 1
    b = ceed.BasisTensorH1Lagrange(1, 1, p + 1, q, libceed.GAUSS)
    interp = b.get_interp_1d().reshape(q, p + 1)
    grad = b.get_grad_1d().reshape(q, p + 1)
    w = np.diag(b.get_q_weight_1d())
    
    M = interp.T @ w @ interp
    K = grad.T   @ w @ grad
    K = K + np.eye(p + 1)*1e-8
    
    X, lam = ceed.simultaneous_diagonalization(K, M, p + 1)
    K = grad.T   @ w @ grad
    X = X.reshape(p + 1, p + 1)

    M_3 = np.kron(np.kron(M, M), M)
    K_3 = np.kron(np.kron(K, M), M) + np.kron(np.kron(M, K), M) + np.kron(np.kron(M, M), K)

    M_inv = np.kron(np.kron(X, X), X) @ np.kron(np.kron(X.T, X.T), X.T)

    ones = np.ones(p + 1)
    lam_3 = np.kron(np.kron(lam, ones), ones) + np.kron(np.kron(ones, lam), ones) + np.kron(np.kron(ones, ones), lam)
    lam_3_pinv = np.diag([1./val for val in lam_3])
    K_pinv = np.kron(np.kron(X, X), X) @ lam_3_pinv @ np.kron(np.kron(X.T, X.T), X.T)

    A = K_3
    A_pinv = K_pinv

    num_vertices = 2**3
    S = np.zeros((num_vertices, num_vertices))
    indx = [0, p, (p + 1)**2 - (p + 1), (p + 1)**2 - 1, (p + 1)**3 - (p + 1)**2, (p + 1)**3 - (p + 1)**2 + p, (p + 1)**3 - (p + 1), (p + 1)**3 - 1]
    for j in range(num_vertices):
        u = np.zeros((p + 1)**3)
        u[indx[j]] = 1
        v = np.matmul(A_pinv, u)
        for i in range(num_vertices):
            S[i, j] = -v[indx[i]]

    S_inv = np.linalg.inv(S)
    
    [nodes, _] = ceed.lobatto_quadrature(p + 1)
    u = np.array([-(node_i - 1.)*(node_i + 1.)-(node_j - 1.)*(node_j + 1.)-(node_k - 1.)*(node_k + 1.) for node_i in nodes for node_j in nodes for node_k in nodes])

    
    v = np.matmul(A, u)
    
    for i in range(num_vertices):
        v[indx[i]] = 0
    w = np.matmul(A_pinv, v)
  
    w_pi = np.zeros(num_vertices)
    for i in range(num_vertices):
        w_pi[i] = w[indx[i]]
    v_pi = np.matmul(S_inv, w_pi)
    for i in range(num_vertices):
        v[indx[i]] = v_pi[i]
    
    w = np.matmul(A_pinv, v)
    
    differences_perturb[p - 1] = np.linalg.norm(u - w)

print(differences_perturb)

In [ ]:
ceed = libceed.Ceed()

max_degree = 8
differences_zero = np.zeros(max_degree)

for p in range(1, max_degree + 1):
    q = p + 1
    b = ceed.BasisTensorH1Lagrange(1, 1, p + 1, q, libceed.GAUSS)
    interp = b.get_interp_1d().reshape(q, p + 1)
    grad = b.get_grad_1d().reshape(q, p + 1)
    w = np.diag(b.get_q_weight_1d())
    
    M = interp.T @ w @ interp
    K = grad.T   @ w @ grad
    
    X, lam = ceed.simultaneous_diagonalization(K, M, p + 1)
    K = grad.T   @ w @ grad
    X = X.reshape(p + 1, p + 1)

    M_3 = np.kron(np.kron(M, M), M)
    K_3 = np.kron(np.kron(K, M), M) + np.kron(np.kron(M, K), M) + np.kron(np.kron(M, M), K)

    M_inv = np.kron(np.kron(X, X), X) @ np.kron(np.kron(X.T, X.T), X.T)

    ones = np.ones(p + 1)
    lam_3 = np.kron(np.kron(lam, ones), ones) + np.kron(np.kron(ones, lam), ones) + np.kron(np.kron(ones, ones), lam)
    lam_3_pinv = np.diag([1./val if abs(val) > 1e-12 else 0 for val in lam_3])
    K_pinv = np.kron(np.kron(X, X), X) @ lam_3_pinv @ np.kron(np.kron(X.T, X.T), X.T)

    A = K_3
    A_pinv = K_pinv

    num_vertices = 2**3
    S = np.zeros((num_vertices, num_vertices))
    indx = [0, p, (p + 1)**2 - (p + 1), (p + 1)**2 - 1, (p + 1)**3 - (p + 1)**2, (p + 1)**3 - (p + 1)**2 + p, (p + 1)**3 - (p + 1), (p + 1)**3 - 1]
    for j in range(num_vertices):
        u = np.zeros((p + 1)**3)
        u[indx[j]] = 1
        v = np.matmul(A_pinv, u)
        for i in range(num_vertices):
            S[i, j] = -v[indx[i]]

    S_inv = np.linalg.inv(S)
    
    [nodes, _] = ceed.lobatto_quadrature(p + 1)
    u = np.array([-(node_i - 1.)*(node_i + 1.)-(node_j - 1.)*(node_j + 1.)-(node_k - 1.)*(node_k + 1.) for node_i in nodes for node_j in nodes for node_k in nodes])

    
    v = np.matmul(A, u)
    
    for i in range(num_vertices):
        v[indx[i]] = 0
    w = np.matmul(A_pinv, v)
  
    w_pi = np.zeros(num_vertices)
    for i in range(num_vertices):
        w_pi[i] = w[indx[i]]
    v_pi = np.matmul(S_inv, w_pi)
    for i in range(num_vertices):
        v[indx[i]] = v_pi[i]
    
    w = np.matmul(A_pinv, v)
    
    differences_zero[p - 1] = np.linalg.norm(u - w)

print(differences_zero)

In [ ]:
# Plot 3D pesudoinverse error
fig, ax1 = plt.subplots(figsize=(10, 8))

plt.xlim([1, 8])

plt.plot(range(1, 9), differences_zero, label='Zeroing', marker='s', markersize=10, color=BLUE)
plt.plot(range(1, 9), differences_perturb, label='Perturbation', marker='o', markersize=10, color=YELLOW)

ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.set_yscale('log')
ax1.grid(color='lightgrey')

plt.legend(fontsize=20)
plt.legend(fontsize=20, loc=[0.63,0.45])
plt.tick_params(labelsize=20)
plt.title('3D Subdomain Solver Error', fontsize=26)
plt.ylabel('2-Norm of Error', fontsize=24)
plt.xlabel('Polynomial Order', fontsize=24)
plt.savefig("subdomainError3D", bbox_inches='tight')